### 목표변수와 설명변수
- Subject: 환자
- Goal: Features가 Target인 수술실패여부에 미치는 영향 분석
- Target: '수술실패여부'
- Features: '고혈압여부', '성별', '신부전여부', '연령', '체중', '수술시간'

In [87]:
import pandas as pd

In [88]:
df_ROS = pd.read_csv('../datasets/RecurrenceOfSurgery.csv')
df_ROS[:2]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [89]:
df_ROS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1894 non-null   int64  
 1   환자ID                    1894 non-null   object 
 2   Large Lymphocyte        1894 non-null   float64
 3   Location of herniation  1894 non-null   int64  
 4   ODI                     462 non-null    float64
 5   가족력                     1843 non-null   float64
 6   간질성폐질환                  1894 non-null   int64  
 7   고혈압여부                   1894 non-null   int64  
 8   과거수술횟수                  1894 non-null   int64  
 9   당뇨여부                    1894 non-null   int64  
 10  말초동맥질환여부                1894 non-null   int64  
 11  빈혈여부                    1894 non-null   int64  
 12  성별                      1894 non-null   int64  
 13  스테로이드치료                 1894 non-null   int64  
 14  신부전여부                   1894 non-null   

### 전처리

In [90]:
df_ROS_select = df_ROS[['수술실패여부', '고혈압여부', '성별', '신부전여부', '연령', '체중', '수술시간']]
df_ROS_select[:2]

,수술실패여부,고혈압여부,성별,신부전여부,연령,체중,수술시간
0,0,0,2,0,66,60.3,68.0
1,0,0,1,0,47,71.7,31.0


#### Scaling & Encoding & Concat

##### - OneHotEncoding

In [91]:
# 범주형 데이터 확인 : '고혈압여부', '성별', '신부전여부'
df_ROS_select['고혈압여부'].value_counts(),df_ROS_select['성별'].value_counts(),df_ROS_select['신부전여부'].value_counts()

(고혈압여부
 0    1646
 1     248
 Name: count, dtype: int64,
 성별
 1    1168
 2     726
 Name: count, dtype: int64,
 신부전여부
 0    1846
 1      48
 Name: count, dtype: int64)

In [92]:
from sklearn.preprocessing import OneHotEncoder

In [93]:
# 범주형 설명변수 OneHotEncoding
oneHotEncoder = OneHotEncoder() # 인스턴스화
oneHotEncoder = oneHotEncoder.fit(df_ROS_select[['고혈압여부', '성별', '신부전여부']])
oneHotEncoder

OneHotEncoder()

In [94]:
oneHotEncoder.categories_

[array([0, 1], dtype=int64),
 array([1, 2], dtype=int64),
 array([0, 1], dtype=int64)]

In [95]:
encoded_data = oneHotEncoder.transform(df_ROS_select[['고혈압여부', '성별', '신부전여부']]).toarray()
encoded_data.shape

(1894, 6)

In [96]:
df_encoded_data = pd.DataFrame(data=encoded_data, columns=oneHotEncoder.get_feature_names_out(['고혈압여부', '성별', '신부전여부']))
df_encoded_data[:2]

,고혈압여부_0,고혈압여부_1,성별_1,성별_2,신부전여부_0,신부전여부_1
0,1.0,0.0,0.0,1.0,1.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0


##### - 병합(Concat)

In [97]:
df_ROS_select= pd.concat([df_ROS_select.reset_index(drop=True), df_encoded_data.reset_index(drop=True)], axis=1)
df_ROS_select[:2]

,수술실패여부,고혈압여부,성별,신부전여부,연령,체중,수술시간,고혈압여부_0,고혈압여부_1,성별_1,성별_2,신부전여부_0,신부전여부_1
0,0,0,2,0,66,60.3,68.0,1.0,0.0,0.0,1.0,1.0,0.0
1,0,0,1,0,47,71.7,31.0,1.0,0.0,1.0,0.0,1.0,0.0


In [98]:
df_ROS_select.shape

(1894, 13)

##### - Scaling

In [99]:
df_ROS_select.columns

Index(['수술실패여부', '고혈압여부', '성별', '신부전여부', '연령', '체중', '수술시간', '고혈압여부_0',
       '고혈압여부_1', '성별_1', '성별_2', '신부전여부_0', '신부전여부_1'],
      dtype='object')

In [100]:
target = df_ROS_select['수술실패여부']
features = df_ROS_select.drop(columns=['수술실패여부', '고혈압여부', '성별', '신부전여부'])

In [101]:
features.columns

Index(['연령', '체중', '수술시간', '고혈압여부_0', '고혈압여부_1', '성별_1', '성별_2', '신부전여부_0',
       '신부전여부_1'],
      dtype='object')

##### - MinMaxScaler

In [102]:
from sklearn.preprocessing import MinMaxScaler

In [103]:
minMaxScaler = MinMaxScaler() #인스턴스화
# features = minMaxSclaer.fit_transform(features)
fit_scaler = minMaxScaler.fit(features)
features_scaler = fit_scaler.transform(features)
features_scaler.shape

(1894, 9)

#### 모델학습, apply()함수를 사용하여 null값 채우기
- null 값 채우기 : '수술시간'

In [104]:
# null값 확인 -> 수술시간 null값 존재
df_ROS_select.isnull().sum()

수술실패여부      0
고혈압여부       0
성별          0
신부전여부       0
연령          0
체중          0
수술시간       54
고혈압여부_0     0
고혈압여부_1     0
성별_1        0
성별_2        0
신부전여부_0     0
신부전여부_1     0
dtype: int64

In [105]:
# null값 삭제 -> null값이 없는 데이터로 모델 학습시키기 위함
df_ROS_select_drop = df_ROS_select.dropna()
df_ROS_select_drop.isnull().sum()

수술실패여부     0
고혈압여부      0
성별         0
신부전여부      0
연령         0
체중         0
수술시간       0
고혈압여부_0    0
고혈압여부_1    0
성별_1       0
성별_2       0
신부전여부_0    0
신부전여부_1    0
dtype: int64

In [106]:
# null값이 없는 데이터로 모델학습 준비
# 1. target : '수술시간', feature: '성별'
target = df_ROS_select_drop[['수술시간']]
feature = df_ROS_select_drop[['성별']]
target.shape, feature.shape

((1840, 1), (1840, 1))

In [107]:
# 2. null값이 없는 데이터와 실제값을 사용하여 회귀모델 훈련
from sklearn.linear_model import LinearRegression
model = LinearRegression() # 인스턴스화(초기화)
model.fit(feature, target) # 모델 훈련

LinearRegression()

In [108]:
# 모델 예측 확인해보기 (type : numpy의 array)
model.predict(feature)

array([[62.40798859],
       [61.85601405],
       [61.85601405],
       ...,
       [61.85601405],
       [61.85601405],
       [62.40798859]])

In [109]:
# apply()
import numpy as np
def convert_notnull(row) :
    if np.isnan(row['수술시간'])  : # 변수 row의 값이 null이라면
        feature = df_ROS_select[['성별']]
        result = model.predict(feature)
        return result[0]
    else :
        return row['수술시간']  # null이 아니면 원래 데이터 값 반환

In [110]:
df_ROS_select['수술시간'] = df_ROS_select.apply(convert_notnull, axis=1)
df_ROS_select['수술시간']

0       68.0
1       31.0
2       78.0
3       73.0
4       29.0
        ... 
1889    80.0
1890    20.0
1891    50.0
1892    25.0
1893    45.0
Name: 수술시간, Length: 1894, dtype: object

In [111]:
# apply() 적용 후 null값 확인
df_ROS_select['수술시간'].isnull().sum()

0

In [112]:
df_ROS_select.isnull().sum()

수술실패여부     0
고혈압여부      0
성별         0
신부전여부      0
연령         0
체중         0
수술시간       0
고혈압여부_0    0
고혈압여부_1    0
성별_1       0
성별_2       0
신부전여부_0    0
신부전여부_1    0
dtype: int64

#### best model 찾기
- hypertuning
- resampling

In [113]:
target = df_ROS_select['수술실패여부']
features = df_ROS_select.drop(columns=['수술실패여부', '고혈압여부', '성별', '신부전여부'])
df_ROS_select.drop(columns=['수술실패여부', '고혈압여부', '성별', '신부전여부']).isnull().sum()

연령         0
체중         0
수술시간       0
고혈압여부_0    0
고혈압여부_1    0
성별_1       0
성별_2       0
신부전여부_0    0
신부전여부_1    0
dtype: int64

##### - HyperTuning

In [114]:
# 기계학습 알고리즘 호출
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [115]:
# resampling 하기 전
from collections import Counter
Counter(df_ROS_select['수술실패여부'])

Counter({0: 1779, 1: 115})

In [116]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()

In [117]:
from sklearn.metrics import make_scorer, f1_score
scoring = make_scorer(f1_score)
hyper_list = {'max_depth':range(2, 10),
              'min_samples_leaf': range(2, 10),
              'criterion': ['gini','entropy'],
              'class_weight': [None, 'balanced'],
              'min_samples_split': range(2, 10)}
grid_model = GridSearchCV(model, param_grid=hyper_list,
                          scoring=scoring,
                          n_jobs = -1,
                          cv = 5)

##### - resampling
- under sampling : NearMiss

In [118]:
df_ROS_select.isnull().sum()

수술실패여부     0
고혈압여부      0
성별         0
신부전여부      0
연령         0
체중         0
수술시간       0
고혈압여부_0    0
고혈압여부_1    0
성별_1       0
성별_2       0
신부전여부_0    0
신부전여부_1    0
dtype: int64

In [119]:
from imblearn.under_sampling import NearMiss
from datetime import datetime

In [120]:
sampling_strategy_list = np.arange(0.3, 1.0, 0.05)     # 0.3~1.0까지 0.05 간격으로 샘플링 전략값을 가지는 리스트

for sampling_strategy in sampling_strategy_list :
    print(30*'-','{}'.format(sampling_strategy))     # 현재 샘플링 전략값에 대한 구분선(------)과 해당값 출력
    resampler = NearMiss(sampling_strategy=sampling_strategy)
    features_under_resample, target_under_resample = resampler.fit_resample(features, target)
    print(Counter(target_under_resample))     # 언더 샘플링 결과 확인
    features_under_resample_train, features_under_resample_test, target_under_resample_train, target_under_resample_test = train_test_split(features_under_resample, target_under_resample, test_size = 0.3, random_state= 1234)     # train-test set분할
    print(Counter(target_under_resample_train), Counter(target_under_resample_test))
    # re learning with oversampling datasets
    
    start = datetime.now()  # start date and time

    grid_model.fit(features_under_resample_train, target_under_resample_train)

    end = datetime.now()  # end date and time - set to current date and time

    duration = end - start  # duration as a timedelta object

    print("Duration: {0}".format(duration))     # 학습에 걸린 시간(duration)출력
    
    best_model = grid_model.best_estimator_     # grid_model에서 best_estimator_속성을 사용해 최적 모델(best_Model) 선택
    target_under_resample_train_pred = best_model.predict(features_under_resample_train)
    target_under_resample_test_pred = best_model.predict(features_under_resample_test)

    print(classification_report(target_under_resample_train, target_under_resample_train_pred))     # 학습 데이터 예측 결과 출력
    print(classification_report(target_under_resample_test, target_under_resample_test_pred))     # 테스트 데이터 예측 결과 출력
    print(30*'-')     # 한 번의 반복이 끝나면 구분선이 출력되어 다음 반복 작업과 구분

------------------------------ 0.3
Counter({0: 383, 1: 115})
Counter({0: 268, 1: 80}) Counter({0: 115, 1: 35})
Duration: 0:00:18.069676
              precision    recall  f1-score   support

           0       0.86      0.91      0.88       268
           1       0.62      0.50      0.56        80

    accuracy                           0.82       348
   macro avg       0.74      0.71      0.72       348
weighted avg       0.81      0.82      0.81       348

              precision    recall  f1-score   support

           0       0.83      0.90      0.86       115
           1       0.54      0.37      0.44        35

    accuracy                           0.78       150
   macro avg       0.68      0.64      0.65       150
weighted avg       0.76      0.78      0.76       150

------------------------------
------------------------------ 0.35
Counter({0: 328, 1: 115})
Counter({0: 230, 1: 80}) Counter({0: 98, 1: 35})
Duration: 0:00:14.446975
              precision    recall  f1-score

#### 정형화

In [121]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.3, random_state = 10)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((1325, 9), (569, 9), (1325,), (569,))

#### 모델학습

In [122]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
model = DecisionTreeClassifier()
# Target이 범주형이므로 Classifier을 사용

In [123]:
from sklearn.model_selection import GridSearchCV
hyper_params = {'min_samples_leaf' : range(2,5),
               'max_depth' : range(2,5),
               'min_samples_split' : range(2,5)}

#### 예측

In [124]:
from sklearn.metrics import f1_score, make_scorer
scoring = make_scorer(f1_score)

In [125]:
grid_search = GridSearchCV(model, param_grid = hyper_params, cv=3, verbose=1, scoring=scoring)
grid_search.fit(features, target)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(2, 5),
                         'min_samples_leaf': range(2, 5),
                         'min_samples_split': range(2, 5)},
             scoring=make_scorer(f1_score), verbose=1)

In [126]:
grid_search.best_estimator_

DecisionTreeClassifier(max_depth=2, min_samples_leaf=2)

In [127]:
grid_search.best_score_, grid_search.best_params_

# 전처리 전의 정확도(accuracy) : 0.028571428571428574
# 낮은 정확도, 모델이 예측을 잘 수행하지 못함

(0.0, {'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 2})

In [128]:
best_model = grid_search.best_estimator_
best_model

DecisionTreeClassifier(max_depth=2, min_samples_leaf=2)

In [129]:
target_test_predict = best_model.predict(features_test)
target_test_predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

#### 평가

In [130]:
from sklearn.metrics import classification_report

In [131]:
print(classification_report(target_test, target_test_predict))

# 전처리 전의 값
#  precision  recall  f1-score   support
#0  0.94      1.00      0.97       516
#1  1.00      0.03      0.05        36

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       535
           1       0.00      0.00      0.00        34

    accuracy                           0.94       569
   macro avg       0.47      0.50      0.48       569
weighted avg       0.88      0.94      0.91       569



C:\Users\hyein\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hyein\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hyein\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 서비스 배포

In [132]:
# 데이터를 저장하고 불러올 때 매우 유용한 라이브러리
# 클래스 자체를 통째로 파일로저장했다가 그것을 그대로 불러올 수 있음
import pickle

In [133]:
# MinMaxScaler class pikle
with open('../datasets/RecurrenceOfSurgery_scaling.pkl','wb') as scaling_file :
    pickle.dump(obj=fit_scaler, file=scaling_file)
    pass

In [134]:
# encoding class pikle
with open('../datasets/RecurrenceOfSurgery_encoding.pkl','wb') as encoding_file :
    pickle.dump(obj=oneHotEncoder, file=encoding_file)
    pass

In [135]:
# best model pikle
# with open('../datasets/RecurrenceOfSurgery_best_model.pkl','wb') as best_model :
#     pickle.dump(obj=,file=best_model)
#     pass